In [1]:
import os
import azureml.core
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import DatabricksStep
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.0


In [2]:
subscription_id = ""
resource_group = ""
workspace_name = ""
databricks_compute_name = ""

In [3]:
ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group)

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
databricks_compute = DatabricksCompute(workspace=ws, name=databricks_compute_name)

# One time setup: Install databricks CLI and configure

In [5]:
!pip install databricks-cli

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
msftkube 1.0.1598662 has requirement applicationinsights==0.11.5, but you'll have applicationinsights 0.11.9 which is incompatible.
msftkube 1.0.1598662 has requirement urllib3==1.22, but you'll have urllib3 1.23 which is incompatible.
azureml-sdk 0.1.0.1060109 has requirement azureml-core==0.1.0.1060109, but you'll have azureml-core 0.1.0.0 which is incompatible.
azureml-sdk 0.1.0.1060109 has requirement azureml-train==0.1.0.1060109, but you'll have azureml-train 0.1.0.0 which is incompatible.
azureml-requirements 0.1.0.888002 has requirement azureml-core==0.1.0.888002, but you'll have azureml-core 0.1.0.0 which is incompatible.
azureml-requirements 0.1.0.888002 has requirement azureml-train==0.1.0.888002, but you'll have azureml-train 0.1.0.0 which is incompatible.
azureml-contrib-widgets 0.1.0.1060109 has requirement azureml-core==0.1.0.1060109, but you'll have azureml-core 0.1.0.0 which is incompatible.
azureml-cont

# Configure CLI

> Run `dbfs configure --token` at command line to setup authentication. You'll need to specify Databricks URL and Personal Access Token. This link has details: https://docs.databricks.com/dev-tools/cli/index.html

# Upload files from local computer to DBFS

In [6]:
local_path = ".\code"
dbfs_path = "dbfs:/UploadTest"

In [ ]:
# Run this if you need to delete existing DBFS folder
# !dbfs rm -r {dbfs_path}

In [14]:
!dbfs cp -r {local_path} {dbfs_path}

.\code\hello.py -> dbfs:/UploadTest/hello.py


# Use the file that was uploaded to DBFS in DatabricksStep

In [15]:
python_script_path = dbfs_path + "/hello.py"  # append name of the entry script
print(python_script_path)

dbfs:/UploadTest/hello.py


In [16]:
dbPythonInDbfsStep = DatabricksStep(
    name="DBPythonInDBFS",
    run_name='DB_Python_demo',
    compute_target=databricks_compute,
    python_script_path=python_script_path,
    python_script_params={'arg1', 'arg2'},
    num_workers=1,
    allow_reuse=True)

In [ ]:
steps = [dbPythonInDbfsStep]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = Experiment(ws, 'DB_Python_demo').submit(pipeline)
pipeline_run.wait_for_completion()